In [4]:
# Library Management System 

# Using python and sql

import os
import sqlite3
# Remove existing database to start fresh
if os.path.exists("lib.db"):
    os.remove("lib.db")


conn = sqlite3.connect("lib.db")
cur = conn.cursor()

# Drop old tables (optional since DB is fresh)
cur.execute("DROP TABLE IF EXISTS issues")
cur.execute("DROP TABLE IF EXISTS borrowers")

# Create tables
cur.execute('''
CREATE TABLE IF NOT EXISTS borrowers (
    borrower_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL
)
''')

cur.execute('''
CREATE TABLE IF NOT EXISTS books (
    book_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT,
    copies INTEGER DEFAULT 1
)
''')

cur.execute('''
CREATE TABLE IF NOT EXISTS issues (
    issue_id INTEGER PRIMARY KEY AUTOINCREMENT,
    borrower_id INTEGER,
    book_id INTEGER,
    issue_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    return_date INTEGER,  -- store 1 if returned, NULL if not
    FOREIGN KEY(borrower_id) REFERENCES borrowers(borrower_id),
    FOREIGN KEY(book_id) REFERENCES books(book_id)
)
''')

conn.commit()

# Preload NCERT Class 12 books into fresh database
books_to_add = [
    ("Physics Part 1 - NCERT Class 12", "NCERT", 3),
    ("Physics Part 2 - NCERT Class 12", "NCERT", 3),
    ("Chemistry - NCERT Class 12", "NCERT", 3),
    ("Mathematics - NCERT Class 12", "NCERT", 3),
    ("Biology - NCERT Class 12", "NCERT", 2)
]
cur.executemany("INSERT INTO books (title, author, copies) VALUES (?, ?, ?)", books_to_add)
conn.commit()

def issue_book(borrower_name, book_id):
    cur.execute("INSERT INTO borrowers (name) VALUES (?)", (borrower_name,))
    borrower_id = cur.lastrowid

    cur.execute("SELECT copies FROM books WHERE book_id = ?", (book_id,))
    book = cur.fetchone()
    if not book:
        print("Book not found.")
        return
    copies = book[0]

    cur.execute('''SELECT COUNT(*) FROM issues 
                   WHERE book_id = ? AND return_date IS NULL''', (book_id,))
    issued_count = cur.fetchone()[0]

    if issued_count >= copies:
        print("No copies available to issue.")
        return

    cur.execute("INSERT INTO issues (borrower_id, book_id) VALUES (?, ?)", (borrower_id, book_id))
    conn.commit()
    print(f"Book ID {book_id} issued to '{borrower_name}' successfully.")

def return_book(borrower_name, book_id):
    cur.execute("SELECT borrower_id FROM borrowers WHERE name = ? ORDER BY borrower_id DESC LIMIT 1", (borrower_name,))
    borrower = cur.fetchone()
    if not borrower:
        print("Borrower not found.")
        return
    borrower_id = borrower[0]

    cur.execute('''SELECT issue_id FROM issues 
                   WHERE borrower_id = ? AND book_id = ? AND return_date IS NULL''', (borrower_id, book_id))
    issue = cur.fetchone()
    if not issue:
        print("No active issue found for this book and borrower.")
        return

    issue_id = issue[0]
    # Instead of datetime, just mark return_date as 1 (means returned)
    cur.execute("UPDATE issues SET return_date = 1 WHERE issue_id = ?", (issue_id,))
    conn.commit()
    print(f"Book ID {book_id} returned by '{borrower_name}' successfully.")

def list_books():
    cur.execute("SELECT book_id, title, author, copies FROM books")
    books = cur.fetchall()
    print("\nAvailable Books:")
    for book_id, title, author, copies in books:
        cur.execute('''SELECT COUNT(*) FROM issues WHERE book_id = ? AND return_date IS NULL''', (book_id,))
        issued = cur.fetchone()[0]
        available = copies - issued
        print(f"ID: {book_id} | Title: {title} | Author: {author} | Available: {available}")

def library_menu():
    while True:
        print("\n===== Library Management System =====")
        print("1. List Books")
        print("2. Issue Book")
        print("3. Return Book")
        print("4. Exit")
        choice = input("Enter your choice: ")

        if choice == '1':
            list_books()
        elif choice == '2':
            borrower_name = input("Enter Borrower Name: ")
            try:
                book_id = int(input("Enter Book ID to issue: "))
                issue_book(borrower_name, book_id)
            except ValueError:
                print("Invalid input! Book ID must be a number.")
        elif choice == '3':
            borrower_name = input("Enter Borrower Name: ")
            try:
                book_id = int(input("Enter Book ID to return: "))
                return_book(borrower_name, book_id)
            except ValueError:
                print("Invalid input! Book ID must be a number.")
        elif choice == '4':
            print("Exiting Library Management System. Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.")

library_menu()
conn.close()


===== Library Management System =====
1. List Books
2. Issue Book
3. Return Book
4. Exit


Enter your choice:  1



Available Books:
ID: 1 | Title: Physics Part 1 - NCERT Class 12 | Author: NCERT | Available: 3
ID: 2 | Title: Physics Part 2 - NCERT Class 12 | Author: NCERT | Available: 3
ID: 3 | Title: Chemistry - NCERT Class 12 | Author: NCERT | Available: 3
ID: 4 | Title: Mathematics - NCERT Class 12 | Author: NCERT | Available: 3
ID: 5 | Title: Biology - NCERT Class 12 | Author: NCERT | Available: 2

===== Library Management System =====
1. List Books
2. Issue Book
3. Return Book
4. Exit


Enter your choice:  2
Enter Borrower Name:  Sneha Rajbhar
Enter Book ID to issue:  3


Book ID 3 issued to 'Sneha Rajbhar' successfully.

===== Library Management System =====
1. List Books
2. Issue Book
3. Return Book
4. Exit


Enter your choice:  3
Enter Borrower Name:  Sneha Rajbhar
Enter Book ID to return:  3


Book ID 3 returned by 'Sneha Rajbhar' successfully.

===== Library Management System =====
1. List Books
2. Issue Book
3. Return Book
4. Exit


Enter your choice:  4


Exiting Library Management System. Goodbye!
